In [1]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Weighted_Raster_Centroid, Raster_BBox
from geometry import Sort_Vertices, Create_Seed
from stippling import Stippler
import time

Rust import

In [2]:
import voronoi_stippling as vs

In [3]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

Image parameters

In [4]:
image_path = 'sampling/blank.jpg'
image_resolution = 1080
image = Image.new('RGB', (image_resolution, image_resolution))
image.save(image_path)
weight_image = Image.open(image_path)
# weight_image = weight_image.resize((size,size))

Class used in stippling for image & weight array

In [5]:
stippler = Stippler(image_path, power=1)
weight_array = stippler.weight_array
print(len(list(weight_array)))

1080


Test polygon

In [6]:
polygon = np.array([[10,10],[150,20],[160,160],[15,150]])
bbox = Raster_BBox(polygon, image_resolution)
rasterized = Scanline_Rasterize_Polygon(polygon,bbox,image_resolution)
t0 = time.time()
centroid = Weighted_Raster_Centroid(rasterized, stippler.weight_array)
t1 = time.time()
print(len(rasterized))

19875


Testing rust function

In [7]:
t2 = time.time()
rust_centroid = vs.weighted_raster_centroid(rasterized, weight_array)
t3 = time.time()
rust_centroid

(83.73688507080078, 85.3984375)

In [11]:
print(f"Python time: {round(t1 - t0, 3)}")
print(f"Rust time: {round(t3 - t2, 3)}")

Python time: 0.033
Rust time: 0.083


Visualizing

In [9]:
scale_factor = 4 # Scale purely for visualizing
image_resolution = image_resolution * scale_factor
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution / 2, image_resolution / 2,image_resolution, image_resolution,'black')

for pixel in rasterized:
    center_rectangle(pixel[0] * scale_factor, pixel[1] * scale_factor, scale_factor/2,scale_factor/2,'white')

center_ellipse(centroid[0] * scale_factor, centroid[1] * scale_factor, 20, 'red')
center_ellipse(rust_centroid[0] * scale_factor, rust_centroid[1] * scale_factor, 15, 'blue')

img.save('rust_centroid.png')